In [11]:
import numpy 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pywt
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold,cross_val_score, cross_validate
from PyEMD import EMD
from scipy import signal
from scipy.stats import differential_entropy,zscore

#4-50Hzでフィルタリング
def filtering(channels, samplerate):   #fsではなくsamplerateとしたのは、阻止域端周波数の変数でfsを使うため
    fp = np.array([4, 50])     #通過域端周波数[Hz]※ベクトル
    fs = np.array([1, 60])      #阻止域端周波数[Hz]※ベクトル
    gpass = 1                       #通過域端最大損失[dB]
    gstop = 10                      #阻止域端最小損失[dB]
    nyquist = samplerate / 2                           #ナイキスト周波数
    wp = fp / nyquist                                  #ナイキスト周波数で通過域端周波数を正規化
    ws = fs / nyquist                                  #ナイキスト周波数で阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  #オーダーとバターワースの正規化周波数を計算
    b, a = signal.butter(N, Wn, "band")           #フィルタ伝達関数の分子と分母を計算
    
    filtered_channels = []
    
    for channel in range(len(channels)):
        y = signal.filtfilt(b, a, channels[channel])                  #信号に対してフィルタをかける
        filtered_channels.append(y)
    
    filtered_channels = np.array(filtered_channels)
    
    #return filtered_channels
    return channels

#標準化:平均0, 標準偏差1
def scaling(channels):
    zscore_chennels = []
    for channel in range(len(channels)):
        zscore_chennels.append(zscore(channels[channel]))
    zscore_chennels = np.array(zscore_chennels)
    
    return zscore_chennels


def extract_features_by_STFT(channels,fs):
    window = np.hanning(625) #ハニング窓
    features=[]
    
    for n in range(len(channels[0])//625):
        features.append([])

    for channel in range(len(channels)):
        start = 0
        end = 625
        # n: 全サンプル数/5sのサンプル数(125Hz*5=625)
        for n in range(len(channels[channel])//625):
            signal_sample = np.array(channels[channel][start:end])
            
            y_fft = np.fft.fft(window * signal_sample) # Fast Fourier Transform
            amp = np.abs(y_fft/(len(signal_sample)/2)) # Amplitude
            flist = np.fft.fftfreq(len(signal_sample), d=1.0/fs) # Frequency List
            amp = amp[1:len(amp)//2] # ナイキスト周波数を考慮
            flist = flist[1:len(flist)//2]
            power = amp**2 # パワースペクトル
            
            theta = np.sum(power[4 *  int((len(flist)+1)//62.5) -1:7 *  int((len(flist)+1)//62.5) ]) #theta
            alpha = np.sum(power[8 *  int((len(flist)+1)//62.5) -1:13 *  int((len(flist)+1)//62.5) ]) #alpha
            beta = np.sum(power[13 *  int((len(flist)+1)//62.5) -1:30 *  int((len(flist)+1)//62.5) ]) #beta
            beta_alpha_ratio = beta/alpha
            theta_beta_ratio = theta/beta
            
            alpha_de = differential_entropy(amp[8 *  int((len(flist)+1)//62.5) -1:13 *  int((len(flist)+1)//62.5) ])
            beta_de = differential_entropy(amp[13 *  int((len(flist)+1)//62.5) -1:30 *  int((len(flist)+1)//62.5) ])
            gamma_de = differential_entropy(amp[30 *  int((len(flist)+1)//62.5) -1:50 *  int((len(flist)+1)//62.5) ])
            alpha_mean = np.mean(amp)
            beta_mean = np.mean(amp)
            gamma_mean = np.mean(amp)
            
            features[n].append(alpha_de) #差分エントロピー
            features[n].append(alpha_mean) #平均
            features[n].append(beta_de)
            features[n].append(beta_mean)
            features[n].append(gamma_de)
            features[n].append(gamma_mean)
            
            features[n].append(beta_alpha_ratio)
            features[n].append(theta_beta_ratio)
            
            start += 625
            end += 625
        
    features = np.array(features)
         
    return features

def extract_features_by_DWT(channels,fs):
    features=[]
    
    for n in range(len(channels[0])//625):
        features.append([])

    for channel in range(len(channels)):
        start = 0
        end = 625
        # n: 全サンプル数/5sのサンプル数(125Hz*5=625)
        for n in range(len(channels[channel])//625):
            signal_sample = np.array(channels[channel][start:end])
            #500Hzのサンプリング周波数なので, 250Hzがナイキストである. よって、levelは5?
            #125Hzには4??
            coeffs = pywt.wavedec(signal_sample,'db4',level = 5)
            #0-8 8-16 16-31 31-63 63-125 125-250
            # 1   2     3     4     5       6
            for i in range(1,4):   #(1,4)
                #plt.plot(coeffs[i+1])
                #plt.title("coefficient"+str(i+1))
                #plt.show()
                
                y_fft = np.fft.fft(coeffs[i]) # Fast Fourier Transform
                amp = np.abs(y_fft/(len(coeffs[i])/2)) # Amplitude
                flist = np.fft.fftfreq(len(coeffs[i]), d=1.0/fs) # Frequency List
                amp = amp[1:len(amp)//2] # ナイキスト周波数を考慮
                flist = flist[1:len(flist)//2]
                
                de = differential_entropy(amp)
                mean = np.mean(amp)
                features[n].append(mean)
                features[n].append(de)
        
            start += 625
            end += 625
        
    features = np.array(features)
        
    return features

#DEを特徴量
def extract_features_by_EMD(channels,fs):
    features=[]
    
    for n in range(len(channels[0])//625):
        features.append([])

    for channel in range(len(channels)):
        start = 0
        end = 625
        # n: 全サンプル数/5sのサンプル数(125Hz*5=625)
        for n in range(len(channels[channel])//625):
            signal_sample = np.array(channels[channel][start:end])          
            IMFs = EMD().emd(signal_sample, None)
            
            for i in range(5):
                #最初の4つのIMFでDEを計算する(5だと段違いで精度高いけども)
                
                y_fft = np.fft.fft(IMFs[i]) # Fast Fourier Transform
                amp = np.abs(y_fft/(len(IMFs[i])/2)) # Amplitude
                flist = np.fft.fftfreq(len(IMFs[i]), d=1.0/fs) # Frequency List
                amp = amp[1:len(amp)//2] # ナイキスト周波数を考慮
                flist = flist[1:len(flist)//2]
                
                de = differential_entropy(amp)
                mean = np.mean(amp)
                features[n].append(mean)
                features[n].append(de)
                
            start += 625
            end += 625

    features = np.array(features)
    
    return features


def print_to_file(accuracy, precision, recall, f1, analysis, classification):
    with open('result_'+analysis+'_'+classification+'.txt', 'w+') as file:
        text = "Accuracy: " + str(accuracy) + "\n"
        text += "Precision: " + str(precision)+ "\n"
        text += "Recall: " + str(recall)+ "\n"
        text += "F1: " + str(f1)+ "\n"
        file.write(text)
            

class Main:
    files = []
    for subject in range(1,6): #被験者5人
        for movie in range(1,4): #動画3つ
            s = 's'
            s += str(subject)
            s += '_'
            s += str(movie)
            files.append(s)
    print(files)
    
    # dataの読み込み
    fs = 125 #サンプリング周波数は500Hzが125Hzにダウンサンプリングされている
    print("dataの読み込み")
    data1=[]  #動画1の読み込み
    data2=[]  #動画2の読み込み
    data3=[]  #動画3の読み込み
    for n,file in enumerate(files):
        filename = file+'.CSV'
        df = pd.read_csv(filename, header=None)  #csvファイルの読み込み
        if (n+1)%3==1:
            print(filename)
            print("data1に追加")
            data1.append(df.iloc[:,0:8].values.T)  #iloc[全部の行, 1列～8列]を取り出してdata1へ
        elif (n+1)%3==2:
            print(filename)
            print("data2に追加")
            data2.append(df.iloc[:,0:8].values.T)  #iloc[全部の行, 1列～8列]を取り出してdata2へ
        elif (n+1)%3==0:
            print(filename)
            print("data3に追加")
            data3.append(df.iloc[:,0:8].values.T)  #iloc[全部の行, 1列～8列]を取り出してdata3へ

    #5人*8チャンネル×11875点(95秒)
    data1 = np.array(data1)
    data2 = np.array(data2)
    data3 = np.array(data3)
    print(len(data1))
    print(len(data1[0]))
    print(len(data1[0][0]))
    print(len(data2[0][0]))
    print(len(data3[0][0]))
    
    #labelの読み込み
    print("labelの読み込み")
    files_label = []
    for subject in range(1,6): #被験者5人
        for movie in range(1,4): #動画3つ
            s = 's'
            s += str(subject)
            s += '_'
            s += str(movie)
            s += '_label'
            files_label.append(s)
    print(files_label)
    
    labels1 = []
    labels2 = []
    labels3 = []
    for n,file in enumerate(files_label):
        filename = file+'.csv'
        df = pd.read_csv(filename, header=None)  #csvファイルの読み込み
        if (n+1)%3==1:
            labels1.append(df.iloc[:,4].values.T)
            #labels1 = np.array(df.iloc[:,0].values.T)  #labels1へ
        elif (n+1)%3==2:
            labels2.append(df.iloc[:,4].values.T)
            #labels2l = np.array(df.iloc[:,0].values.T)  #labels2へ
        elif (n+1)%3==0:
            labels3.append(df.iloc[:,4].values.T)
            #labels3 = np.array(df.iloc[:,0].values.T)  #labels3へ
    
    #arrayにしてから1次元に変換
    labels1 = np.array(labels1)
    labels1 = labels1.ravel()
    labels2 = np.array(labels2)
    labels2 = labels2.ravel()
    labels3 = np.array(labels3)
    labels3 = labels3.ravel()
    print(len(labels3))
    
    labels = []
    for i in range(325):#325
        labels.append([])
        
    for n,label in enumerate(labels1):
        labels[n].append(label)
    for n,label in enumerate(labels2):
        labels[n+95].append(label)  #19*~  95
    for n,label in enumerate(labels3):
        labels[n+170].append(label)  #34*~  170
    
    labels = np.array(labels)
    labels = labels.ravel()  #次元を1次元に
    print(len(labels))
    print(labels)

    #Filtering 動画ごとに
    print("フィルタリング 4-50Hz")
    filtered_data1 = []
    for subject in range(len(data1)):
        #各被験者のデータごとにフィルタリング
        filtered_data1.append(filtering(data1[subject],samplerate=fs))

    filtered_data2 = []
    for subject in range(len(data2)):
        #各被験者のデータごとにフィルタリング
        filtered_data2.append(filtering(data2[subject],samplerate=fs))

    filtered_data3 = []
    for subject in range(len(data3)):
        #各被験者のデータごとにフィルタリング
        filtered_data3.append(filtering(data3[subject],samplerate=fs))
    
    filtered_data1 = np.array(filtered_data1)
    filtered_data2 = np.array(filtered_data2)
    filtered_data3 = np.array(filtered_data3)
    
    #Scaling 標準化
    print("標準化 Z-score")
    stand_data1=[]
    for subject in range(len(filtered_data1)):
        stand_data1.append(scaling(filtered_data1))
    stand_data2=[]
    for subject in range(len(filtered_data2)):
        stand_data2.append(scaling(filtered_data2))
    stand_data3=[]
    for subject in range(len(filtered_data3)):
        stand_data3.append(scaling(filtered_data3))
        
    stand_data1=np.array(stand_data1)
    stand_data2=np.array(stand_data2)
    stand_data3=np.array(stand_data3)
    #stand_dataたちは使ってません。
        
    
    #Extracting features 動画ごとに
    print("特徴量抽出")
    features_data1_STFT = []
    features_data1_DWT = []
    features_data1_EMD = []
    for subject in range(len(filtered_data1)):
        features_data1_STFT.append(extract_features_by_STFT(filtered_data1[subject],fs))
        features_data1_DWT.append(extract_features_by_DWT(filtered_data1[subject],fs))
        features_data1_EMD.append(extract_features_by_EMD(filtered_data1[subject],fs))
    
    features_data2_STFT = []
    features_data2_DWT = []
    features_data2_EMD = []
    for subject in range(len(filtered_data2)):
        features_data2_STFT.append(extract_features_by_STFT(filtered_data2[subject],fs))
        features_data2_DWT.append(extract_features_by_DWT(filtered_data2[subject],fs))
        features_data2_EMD.append(extract_features_by_EMD(filtered_data2[subject],fs))
    
    features_data3_STFT = []
    features_data3_DWT = []
    features_data3_EMD = []
    for subject in range(len(filtered_data3)):
        features_data3_STFT.append(extract_features_by_STFT(filtered_data3[subject],fs))
        features_data3_DWT.append(extract_features_by_DWT(filtered_data3[subject],fs))
        features_data3_EMD.append(extract_features_by_EMD(filtered_data3[subject],fs))

    #Making Training data
    X_STFT = [] # STFT用
    for subject in range(len(features_data1_STFT)):
        for f in features_data1_STFT[subject]:  #セグメントごとに追加
            X_STFT.append(f)
    for subject in range(len(features_data2_STFT)):
        for f in features_data2_STFT[subject]:
            X_STFT.append(f)
    for subject in range(len(features_data3_STFT)):
        for f in features_data3_STFT[subject]:
            X_STFT.append(f)
    X_STFT = np.array(X_STFT)
    
    X_DWT = [] # DWT用
    for subject in range(len(features_data1_DWT)):
        for f in features_data1_DWT[subject]:  #セグメントごとに追加
            X_DWT.append(f)
    for subject in range(len(features_data2_DWT)):
        for f in features_data2_DWT[subject]:
            X_DWT.append(f)
    for subject in range(len(features_data3_DWT)):
        for f in features_data3_DWT[subject]:
            X_DWT.append(f)
    X_DWT = np.array(X_DWT)
    
    X_EMD = [] # EMD用
    for subject in range(len(features_data1_EMD)):
        for f in features_data1_EMD[subject]:  #セグメントごとに追加
            X_EMD.append(f)
    for subject in range(len(features_data2_EMD)):
        for f in features_data2_EMD[subject]:
            X_EMD.append(f)
    for subject in range(len(features_data3_EMD)):
        for f in features_data3_EMD[subject]:
            X_EMD.append(f)
    X_EMD = np.array(X_EMD)
        
    # classification by SVM
    print("分類")
    #5 fold cross validation
    kf = KFold(n_splits=5, shuffle=True, random_state=2)#split5
    
    scoring = {
        'accuracy': 'accuracy',
        'precision': 'precision',
        'recall' :'recall',
        'f1': 'f1',
    } 
    
    """
    STFT
    """
    print("STFTによる分類")
    svm = SVC(kernel='rbf', C=1, gamma=0.1, class_weight='balanced')
    scores = cross_validate(svm, X_STFT, labels, cv=kf, scoring=scoring)
    print("SVM")
    print("accuracy: {}".format(np.mean(scores['test_accuracy'])))
    print("precision: {}".format(np.mean(scores['test_precision'])))
    print("recall: {}".format(np.mean(scores['test_recall'])))
    print("f1: {}\n".format(np.mean(scores['test_f1'])))
    print_to_file(np.mean(scores['test_accuracy']), np.mean(scores['test_precision']), np.mean(scores['test_recall']),
                  np.mean(scores['test_f1']), "STFT", "SVM")
    
    """
    DWT
    """
    print("DWTによる分類")
    svm = SVC(kernel='rbf', C=1, gamma=0.1, class_weight='balanced')
    scores = cross_validate(svm, X_DWT, labels, cv=kf, scoring=scoring)
    print("SVM")
    print("accuracy: {}".format(np.mean(scores['test_accuracy'])))
    print("precision: {}".format(np.mean(scores['test_precision'])))
    print("recall: {}".format(np.mean(scores['test_recall'])))
    print("f1: {}\n".format(np.mean(scores['test_f1'])))
    print_to_file(np.mean(scores['test_accuracy']), np.mean(scores['test_precision']), np.mean(scores['test_recall']),
                  np.mean(scores['test_f1']), "DWT", "SVM")
    
    """
    EMD
    """
    print("EMDによる分類")
    svm = SVC(kernel='rbf', C=1, gamma=0.1, class_weight='balanced')
    scores = cross_validate(svm, X_EMD, labels, cv=kf, scoring=scoring)
    print("SVM")
    print("accuracy: {}".format(np.mean(scores['test_accuracy'])))
    print("precision: {}".format(np.mean(scores['test_precision'])))
    print("recall: {}".format(np.mean(scores['test_recall'])))
    print("f1: {}\n".format(np.mean(scores['test_f1'])))
    print_to_file(np.mean(scores['test_accuracy']), np.mean(scores['test_precision']), np.mean(scores['test_recall']),
                  np.mean(scores['test_f1']), "EMD", "SVM")
    """
    # test
    from sklearn.metrics import accuracy_score
    svm_test_STFT = SVC(kernel='rbf', C=1, gamma=0.1, class_weight='balanced')
    svm_test_STFT.fit(X_STFT[:294], labels[:294])
    y_pred = svm_test_STFT.predict(X_STFT[294:325])
    accuracy_score(labels[294:325], y_pred)
    print("Predicted values: " + str(y_pred))
    print("Real values:      " + str(labels[294:325]))
    print(str(accuracy_score(labels[294:325], y_pred)))
    print("\n")
    
    svm_test_EMD = SVC(kernel='rbf', C=1, gamma=0.1, class_weight='balanced')
    svm_test_EMD.fit(X_EMD[:294], labels[:294])
    y_pred = svm_test_EMD.predict(X_EMD[294:325])
    accuracy_score(labels[294:325], y_pred)
    print("Predicted values: " + str(y_pred))
    print("Real values:      " + str(labels[294:325]))
    print(str(accuracy_score(labels[294:325], y_pred)))"""

['s1_1', 's1_2', 's1_3', 's2_1', 's2_2', 's2_3', 's3_1', 's3_2', 's3_3', 's4_1', 's4_2', 's4_3', 's5_1', 's5_2', 's5_3']
dataの読み込み
s1_1.CSV
data1に追加
s1_2.CSV
data2に追加
s1_3.CSV
data3に追加
s2_1.CSV
data1に追加
s2_2.CSV
data2に追加
s2_3.CSV
data3に追加
s3_1.CSV
data1に追加
s3_2.CSV
data2に追加
s3_3.CSV
data3に追加
s4_1.CSV
data1に追加
s4_2.CSV
data2に追加
s4_3.CSV
data3に追加
s5_1.CSV
data1に追加
s5_2.CSV
data2に追加
s5_3.CSV
data3に追加
5
8
11875
9375
19375
labelの読み込み
['s1_1_label', 's1_2_label', 's1_3_label', 's2_1_label', 's2_2_label', 's2_3_label', 's3_1_label', 's3_2_label', 's3_3_label', 's4_1_label', 's4_2_label', 's4_3_label', 's5_1_label', 's5_2_label', 's5_3_label']
155
325
[1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1
 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0
 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 0 1 0 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1